In [ ]:
%%time
import gc

import numpy as np
import pandas as pd
from pathlib import Path

# from pandarallel import pandarallel
# pandarallel.initialize()

BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting/')
#print(-1)
train = pd.read_csv(BASE_DIR / 'train_updated.csv')
#print(0)
null = np.nan
true = True
false = False

for col in ["rosters", "nextDayPlayerEngagement", "playerBoxScores"]:

    if col == 'date': continue

    _index = train[col].notnull()
    train.loc[_index, col] = train.loc[_index, col].apply(lambda x: eval(x))

    outputs = []
    #print(1)
    for index, date, record in train.loc[_index, ['date', col]].itertuples():
        _df = pd.DataFrame(record)
        _df['index'] = index
        _df['date'] = date
        outputs.append(_df)
    #print(2)
    outputs = pd.concat(outputs).reset_index(drop=True)
    if col == "rosters":
        rosters = outputs.copy()
    elif col == "nextDayPlayerEngagement":
        targets = outputs.copy()
    elif col == "playerBoxScores":
        scores = outputs.copy()
    

    del outputs, _index, _df
    del train[col]
    gc.collect()

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2  
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)  
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
players = pd.read_csv(BASE_DIR / 'players.csv')
scores['positionCode'] = scores['positionCode'].astype(int)
scores = scores.groupby(['playerId', 'date']).sum().reset_index()

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
from functools import reduce
from tqdm import tqdm
import lightgbm as lgbm

import mlb

In [ ]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date', 'positionCode']

feature_cols = [
    'label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status',
    'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
    'positionCode'
]


In [ ]:
player_target_stats = pd.read_csv("../input/player-target-stats/player_target_stats.csv")
data_names=player_target_stats.columns.values.tolist()

In [ ]:
# creat dataset
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(player_target_stats, how='left', left_on=["playerId"],right_on=["playerId"])

In [ ]:
# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)

In [ ]:
seasons_df = pd.read_csv(BASE_DIR / 'seasons.csv')
seasons_df['seasonStartDate'] = pd.to_datetime(seasons_df['seasonStartDate'], format='%Y-%m-%d')
all_cols = train.columns.values.tolist()
train['year'] = pd.to_datetime(train['date'], format = '%Y%m%d').dt.year
train = pd.merge(train,
                   seasons_df,
                   how = 'left',
                   left_on = 'year',
                   right_on = 'seasonId')
train['days_to_season_start'] = (train['seasonStartDate'] - pd.to_datetime(train['date'], format = '%Y%m%d')).dt.days
all_cols.append('days_to_season_start')
feature_cols.append('days_to_season_start')
train = train[all_cols]

In [ ]:
train["isGame"] = (train.positionCode.notna()).astype(np.int32)

In [ ]:
from tqdm import tqdm

train["daysAfterGame"] = np.nan
train["totalGames"] = np.nan

for player in tqdm(train.playerId.unique()):
    player_mask = train.playerId == player
    result = []
    result_count = []
    value = 0
    count = 0
    for i in train.loc[player_mask, "isGame"]:
        if i == 1:
            count += 1
            result.append(0)
            result_count.append(count)
            value = 0
        else:
            value += 1
            result.append(value)
            result_count.append(count)
    train.loc[player_mask, "daysAfterGame"] = result
    train.loc[player_mask, "totalGames"] = result_count

In [ ]:
train['year'] = pd.to_datetime(train['date'], format = '%Y%m%d').dt.year
train['month'] = pd.to_datetime(train['date'], format = '%Y%m%d').dt.month

In [ ]:
dfs = []
for start, end in zip(seasons_df.seasonStartDate.astype(str)[1:], seasons_df.seasonEndDate.astype(str)[1:]):
    start = int(start.replace("-", ""))
    end = int(end.replace("-", ""))
    
    mask = np.logical_and(
        train.date >= start,
        train.date <= end
    )
    
    dfs.append(
        train[mask]
    
    )
train = pd.concat(dfs, axis=0)

In [ ]:
feature_cols += ["daysAfterGame", 'isGame']


In [ ]:
train["rankMonth"] = train.groupby("year").month.transform("rank", method="dense")
train_stats = train[train.rankMonth.isin([3, 4, 5, 6])].reset_index(drop=True)

stats = train_stats.groupby(["playerId", "year"])[[
        "target1", "target2", "target3", "target4"
    ]].agg(
    ["mean", "std", "median", "max", "min"]
)


stats.columns = [f"3year_{c[1]}_{c[0]}"for c in stats.columns]
train = train.merge(stats.reset_index(), how="left", on=["playerId", "year"])

In [ ]:
feature_cols += stats.columns.tolist()

In [ ]:
train["rankMonth"] = train.groupby("year").month.transform("rank", method="dense")
train_stats = train[train.rankMonth.isin([3, 4, 5, 6])].reset_index(drop=True)

stats_team = train_stats.groupby(["teamId", "year"])[[
        "target1", "target2", "target3", "target4"
    ]].agg(
    ["mean", "std", "median", "max", "min"]
)


stats_team.columns = [f"3year_{c[1]}_{c[0]}_teamId" for c in stats_team.columns]
train = train.merge(stats_team.reset_index(), how="left", on=["teamId", "year"])

In [ ]:
feature_cols += stats_team.columns.tolist()


In [ ]:
train["rankMonth"] = train.groupby("year").month.transform("rank", method="dense")
train["shiftIsGame"] = train.groupby("playerId").isGame.transform("shift", periods=-1)
train_stats = train[train.rankMonth.isin([3, 4, 5, 6])].reset_index(drop=True)
train_stats = train_stats[train_stats.shiftIsGame == 1].reset_index(drop=True)


stats_code = train_stats.groupby(["year", "playerId"])[[
        "target1", "target2", "target3", "target4"
    ]].agg(
    ["mean", "std", "median", "max", "min"]
)


stats_code.columns = [f"3year_{c[1]}_{c[0]}_shiftIsGame1" for c in stats_code.columns]
train = train.merge(stats_code.reset_index(), how="left", on=["year", "playerId"])

In [ ]:
feature_cols += stats_code.columns.tolist()


In [ ]:
train["rankMonth"] = train.groupby("year").month.transform("rank", method="dense")
train_stats = train[train.rankMonth.isin([6])].reset_index(drop=True)

stats1 = train_stats.groupby(["playerId", "year"])[[
        "target1", "target2", "target3", "target4"
    ]].agg(
    ["mean", "std", "median", "max", "min"]
)


stats1.columns = [f"3year_{c[1]}_{c[0]}_1"for c in stats1.columns]
train = train.merge(stats1.reset_index(), how="left", on=["playerId", "year"])

In [ ]:
feature_cols += stats1.columns.tolist()
feature_cols += ["year", "rankMonth"]

In [ ]:
train["rankMonth"] = train.groupby("year").month.transform("rank", method="dense")
train_stats = train[train.rankMonth.isin([5, 6])].reset_index(drop=True)

stats2 = train_stats.groupby(["playerId", "year"])[[
        "target1", "target2", "target3", "target4"
    ]].agg(
    ["mean", "std", "median", "max", "min"]
)


stats2.columns = [f"3year_{c[1]}_{c[0]}_2" for c in stats2.columns]
train = train.merge(stats2.reset_index(), how="left", on=["playerId", "year"])

In [ ]:
feature_cols += stats2.columns.tolist()


In [ ]:
# train = train[train.date < 20210501].reset_index(drop=True)  # use all data

In [ ]:
train = train[train.playerId.isin(players[players.playerForTestSetAndFuturePreds.fillna(False)].playerId)]

In [ ]:
train = train.reset_index(drop=True)

In [ ]:
def use_reduce_memory_usage():
    datasets = [train_X, train_y, x_train1, y_train1, x_valid1, y_valid1,
                x_train2, y_train2, x_valid2, y_valid2]

    for data in datasets:
        reduce_mem_usage(data)

In [ ]:
from sklearn.model_selection import GroupKFold

In [ ]:
def fit_lgbm_(
    train, features, target, params

):
    params["n_estimators"] = 5000
    models = []
    oof = np.zeros(train.shape[0])
    cv = GroupKFold(n_splits=4)
    # for random_state in [2344, 33, 768, 42]:
    for train_idx, valid_idx in cv.split(train, train, groups=train.year):

        X_train, X_valid = train.loc[train_idx, features], train.loc[valid_idx, features]
        y_train, y_valid = train.loc[train_idx, target], train.loc[valid_idx, target]

        model = lgbm.LGBMRegressor(**params, n_jobs=-1,
                                   # random_state=random_state
                                   )

        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100
        )

        models.append(model)
        oof[valid_idx] = model.predict(X_valid)
    return models, oof

In [ ]:
feature_cols = list(set(feature_cols) - set(['hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']))

In [ ]:
params1 = {'objective':'mae','reg_alpha': 0.14947461820098767, 'reg_lambda': 0.10185644384043743,
           'learning_rate': 0.08046301304430488, 'num_leaves': 674, 'feature_fraction': 0.9101240539122566,
           'bagging_fraction': 0.9884451442950513, 'bagging_freq': 8, 'min_child_samples': 51}

models1_, oof1_ = fit_lgbm_(
    train, feature_cols, "target1", params1
)

In [ ]:
params2 = {'bagging_fraction': 0.87, 'boosting': 'gbdt', 'feature_fraction': 0.58, 'lambda_l1': 4.572164206128591,
          'lambda_l2': 2.5326666967819307, 'learning_rate': 0.03180599834934879, 'max_bin': 170, 'max_depth': 7,
          'metric': 'mae', 'min_data_in_bin': 43, 'min_data_in_leaf': 205, 'min_gain_to_split': 0.35000000000000003,
          'num_leaves': 119, 'objective': 'mae', 'subsample': 0.8363057030853551}

models2_, oof2_ = fit_lgbm_(
    train, feature_cols, "target2", params2
)

In [ ]:
params3 = {'bagging_fraction': 0.63, 'boosting': 'gbdt', 'feature_fraction': 0.8, 'lambda_l1': 2.5188543779297663,
          'lambda_l2': 1.1247539949521028, 'learning_rate': 0.022110756156901078, 'max_bin': 217, 'max_depth': 7,
          'metric': 'mae', 'min_data_in_bin': 165, 'min_data_in_leaf': 169, 'min_gain_to_split': 2.47, 'num_leaves': 128,
          'objective': 'mae', 'subsample': 0.6468413409317881}
models3_, oof3_ = fit_lgbm_(
    train, feature_cols, "target3", params3
)

In [ ]:
params4 = {'bagging_fraction': 0.53, 'boosting': 'gbdt', 'feature_fraction': 0.5, 'lambda_l1': 2.3658020794556784,
           'lambda_l2': 1.36597241600689, 'learning_rate': 0.11612709577750321, 'max_bin': 150, 'max_depth': 13,
           'metric': 'mae', 'min_data_in_bin': 89, 'min_data_in_leaf': 223, 'min_gain_to_split': 0.63, 'num_leaves': 86, 
           'objective': 'fair', 'subsample': 0.8314126324024551}
models4_, oof4_ = fit_lgbm_(
    train, feature_cols, "target4", params4
)

In [ ]:
player_map = dict()

for player in tqdm(train.playerId.unique()):
    value = train.daysAfterGame[train.playerId == player].values[-1]
    player_map[player] = value

In [ ]:
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', "positionCode"
]

null = np.nan
true = True
false = False

In [ ]:
rank_dict = {
    4: 3,
    5: 4,
    6: 5,
    7: 6,
    8: 7,
    9: 8,
    10: 9
    
}

In [ ]:
import copy

# mlb.make_env.__called__ = False
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in tqdm(iter_test): # make predictions here
    
    sub = copy.deepcopy(sample_prediction_df.reset_index())
    sample_prediction_df = copy.deepcopy(sample_prediction_df.reset_index(drop=True))
    
    # LGBM summit
    # creat dataset
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
    if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
        test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
        test_scores["positionCode"] = test_scores["positionCode"].astype(np.int32)
    else:
        test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in scores.columns:
            if col == 'playerId': continue
            test_scores[col] = np.nan
    test_scores = test_scores.groupby('playerId').sum().reset_index()
    test = sample_prediction_df[['playerId']].copy()
    test = test.merge(players[players_cols], on='playerId', how='left')
    test = test.merge(test_rosters[rosters_cols], on='playerId', how='left')
    test = test.merge(test_scores[scores_cols], on='playerId', how='left')
    test = test.merge(player_target_stats, how='left', left_on=["playerId"],right_on=["playerId"])
    

    test['label_playerId'] = test['playerId'].map(player2num)
    test['label_primaryPositionName'] = test['primaryPositionName'].map(position2num)
    test['label_teamId'] = test['teamId'].map(teamid2num)
    test['label_status'] = test['status'].map(status2num)
    
    # print(test.shape)    
    all_cols = test.columns.values.tolist()
    test['year'] = 2021
    test = pd.merge(test,
                       seasons_df,
                       how = 'left',
                       left_on = 'year',
                       right_on = 'seasonId')
    test['days_to_season_start'] = (test['seasonStartDate'] - pd.to_datetime(test_df.index.values[0], format = '%Y%m%d')).dt.days
    all_cols.append('days_to_season_start')
    test = test[all_cols]   
    # make daysAfterGame
    test["isGame"] = test["positionCode"].notna().astype(np.int32)
    
    for player, value in player_map.items():
        if player in test.playerId.unique():
            is_game = test.isGame[test.playerId == player].values[-1]
            if is_game == 1:
                player_map[player] = 0
            elif is_game == 0:
                player_map[player] = value + 1
        else:
            player_map[player] = value + 1
    
    test_ = pd.DataFrame()
    
    test_["playerId"] = list(player_map.keys())
    test_["daysAfterGame"] = list(player_map.values())
    
    test = test.merge(test_, how="left", on="playerId")
    # make daysAfterGame
    test['year'] = 2021
    test = test.merge(stats.reset_index(), how="left", on=["playerId", "year"])
    test = test.merge(stats_team.reset_index(), how="left", on=["teamId", "year"])
    test = test.merge(stats_code.reset_index(), how="left", on=["playerId", "year"])
    test["rankMonth"] = rank_dict[pd.to_datetime(test_df.index.values[0], format = '%Y%m%d').month]
    test = test.merge(stats1.reset_index(), how="left", on=["playerId", "year"])
    test = test.merge(stats2.reset_index(), how="left", on=["playerId", "year"])
        
    test_X = test[feature_cols]

    pred1_lgb = []
    for model in models1_:
        pred1_lgb.append(model.predict(test_X))
    pred1_lgb = np.mean(pred1_lgb, axis=0)
    
    pred2_lgb = []
    for model in models2_:
        pred2_lgb.append(model.predict(test_X))
    pred2_lgb = np.mean(pred2_lgb, axis=0)
    
    
    pred3_lgb = []
    for model in models3_:
        pred3_lgb.append(model.predict(test_X))
    pred3_lgb = np.mean(pred3_lgb, axis=0)
    
    pred4_lgb = []
    for model in models4_:
        pred4_lgb.append(model.predict(test_X))
    pred4_lgb = np.mean(pred4_lgb, axis=0)

    

    sample_prediction_df['target1'] = np.clip(pred1_lgb, 0, 100)
    sample_prediction_df['target2'] = np.clip(pred2_lgb, 0, 100)
    sample_prediction_df['target3'] = np.clip(pred3_lgb, 0, 100)
    sample_prediction_df['target4'] = np.clip(pred4_lgb, 0, 100)
    sample_prediction_df = sample_prediction_df.fillna(0.)
    del sample_prediction_df['playerId']

    env.predict(sample_prediction_df)
